# import libraries

In [ ]:
import csv
import cv2
import geopandas
import math
import matplotlib.pyplot
import numpy
import os
import pandas

from esda import Moran, Geary
from ipywidgets import interact, interactive, fixed, interact_manual
from libpysal.weights import Queen
from scipy import ndimage
from scipy.spatial import cKDTree, Voronoi, voronoi_plot_2d,distance
from scipy.spatial.distance import cdist
from shapely.geometry import Point
from skimage import restoration, data, img_as_float
from skimage.feature import peak_local_max
from skimage.segmentation import watershed

# Functions

In [ ]:
def Showim(image,imlistCon,showing=False,ImageSaveFolder="",cmap="plasma"):
    if showing==True:
        matplotlib.pyplot.imshow(image,cmap)
        matplotlib.pyplot.axis('off')
        if ImageSaveFolder != "":
            matplotlib.pyplot.savefig(ImageSaveFolder+"/"+imlistCon.split("\\")[-1].split("/")[-1].split(".")[0]+".png")
        matplotlib.pyplot.show()

def countArray(DArray,value):
    #DArray=labels
    #value=ID
    x=0
    for x1 in range(len(DArray)):
        for x2 in range(len(DArray[x1])):
            if DArray[x1][x2]==value:
                x=x+1
    return(x)

#Process image 1
def backgroundrolling(im,rad,imlistCon,fn=0,showing=False,ImageSaveFolder=""):
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) #threshold
    #print("Dye")
    #Showim(imgray,"/Dye"+imlistCon[fn],showing,ImageSaveFolder)
    ####
    background = restoration.rolling_ball(imgray, radius=rad)
    #print("Background")
    #Showim(background,"/backgroundDye-"+imlistCon[fn],showing,ImageSaveFolder)
    imgray2=imgray-background
    imgray3=imgray-background
    if showing==True:
        print("Background subtracted")
        Showim(imgray2,"/subtractedBackgroundDye-"+imlistCon[fn],showing,ImageSaveFolder)
    return(imgray,imgray2,imgray3)

def FindLocalMax(imgray2,size_value,min_distance_value,threshold_abs_value,edge_margin=50,showing=False):
    im = img_as_float(imgray2)
    # image_max is the dilation of im with a 20*20 structuring element
    # It is used within peak_local_max function
    image_max = ndimage.maximum_filter(im, size=size_value, mode='constant')
    # Comparison between image_max and im to find the coordinates of local maxima
    coordinates = peak_local_max(im, min_distance=min_distance_value, threshold_abs=threshold_abs_value,exclude_border=True)
    height, width = im.shape
    # Filter out points near the edges (within 10 pixels)
    coordinates = numpy.array([coord for coord in coordinates 
                                     if edge_margin <= coord[0] < height - edge_margin and 
                                        edge_margin <= coord[1] < width - edge_margin])
    # display results
    if showing==True:
        print("coordinates peaks")
        matplotlib.pyplot.imshow(im, cmap=matplotlib.pyplot.cm.gray)
        matplotlib.pyplot.plot(coordinates[:, 1], coordinates[:, 0], 'r.')
        matplotlib.pyplot.axis('off')
        matplotlib.pyplot.show()
    return(image_max,coordinates)

#segmentar + watershed
def threshold(imgray2,imlistCon,fn,thr_value2,Blocksize=15,Constant=-1,edge_margin=15,filtered_image=None,showing=True,ImageSaveFolder=""):
    if showing==True:
        print("thresholded image")
    #ret, thresh = cv2.threshold(imgray2, thr_value,thr_value2,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    imgray2 = cv2.GaussianBlur(imgray2,(5,5),0)
    thresh = cv2.adaptiveThreshold(imgray2,thr_value2,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,Blocksize,Constant)
    kernel = numpy.ones((3, 3), numpy.uint8)  # Kernel for dilation/erosion
    thresh = cv2.erode(thresh, kernel, iterations=1)  # Erosion to reduce noise
    thresh = cv2.dilate(thresh, kernel, iterations=1)  # Dilation to expand blobs
    thresh_with_border = numpy.zeros_like(imgray2)
    thresh_with_border[edge_margin:-edge_margin, edge_margin:-edge_margin] = thresh[edge_margin:-edge_margin, edge_margin:-edge_margin]
    if not isinstance(filtered_image, numpy.ndarray):
        result_image=thresh_with_border
    else:
        result_image = cv2.bitwise_and(filtered_image, thresh_with_border)
    Showim(result_image,"/thresholdDye-"+imlistCon[fn],showing,ImageSaveFolder)
    return(result_image)


def coords(imlistCon,fn,thresh,coordinates,imgray3,min_thresh_counter=4,showing=True,ImageSaveFolder=""):
    # Now we want to separate the two objects in image
    # Generate the markers as local maxima of the distance to the background
    distance = ndimage.distance_transform_edt(thresh)
    mask = numpy.zeros(distance.shape, dtype=bool)
    mask[tuple(coordinates.T)] = True
    markers, _ = ndimage.label(mask)
    labels = watershed(-distance, markers, mask=thresh)
    
    # Count occurrences of each label
    unique, counts = numpy.unique(labels, return_counts=True)
    label_counts = dict(zip(unique, counts))

    # Remove labels with less than `min_thresh_counter` pixels
    under_thresh = {label for label, count in label_counts.items() if count < min_thresh_counter}
    labels[numpy.isin(labels, list(under_thresh))] = 0  # Vectorized filtering
    nlabels = len(label_counts) - len(under_thresh)
    
    # Get the original 'prism' colormap
    original_cmap = matplotlib.pyplot.cm.prism
    
    # Create a new colormap that has black for label 0 and the rest from 'prism'
    cmap_list = [ (0, 0, 0)] + [original_cmap(i) for i in range(1, 250)]  # Black for 0, rest from 'prism'
    new_cmap = matplotlib.colors.ListedColormap(cmap_list)
    if showing==True:
        # Plot using the new colormap
        matplotlib.pyplot.imshow(labels, cmap=new_cmap)
        print("filtered watershed")
        Showim(labels,"/threholdedFilteredWatershed-"+imlistCon[fn],showing,ImageSaveFolder,cmap=new_cmap)
    
    # Initialize lists to hold filtered coordinates, areas, and IDs
    Areas, IDs, filtered_coordinates,xs,ys = [], [], [],[],[]
    contour_list = []
    Centers,Ferets,angles=[],[],[]
    # Iterate over coordinates to filter them based on their label
    for n in range(len(coordinates)):
        # Get the label at the current coordinate
        y, x = coordinates[n] # y, x because coordinates are (row, col)

        #print("coordinates are ("+str(x)+","+str(y)+")")
        label = labels[y, x] 
        
        # If the label is valid (not 0) and the area of the label is above the threshold
        if label != 0 and label not in under_thresh:
            ID=label
            result = labels == ID
            result = (result * 255).astype(numpy.uint8)
            contours, _ = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contour_list.append(contours)
            if 5>len(contours[0]):
                circle = cv2.minEnclosingCircle(contours[0])
                (xc,yc),rad = circle
                (major,minor) = (2*rad,2*rad)
                angle=0.0
            else:
                ellipse = cv2.fitEllipse(contours[0])
                (xc,yc),(major, minor),angle = ellipse
            if numpy.isfinite(xc) and numpy.isfinite(yc):
                IDs.append(label)
                Area = countArray(labels, label)
                Areas.append(Area)
                filtered_coordinates.append(coordinates[n])
                ys.append(y)
                xs.append(x)
                Centers.append((xc,yc))
                Ferets.append((major, minor))
                angles.append(angle)
                #print("coords :"+str((xc,yc))+"; ferets: "+str((major, minor))+" angle: "+str(angle)) 
                #print()
        elif label == 0:
            continue
    
    #Centers=[(x, y) for x, y in Centers if numpy.isfinite(x) and numpy.isfinite(y)]
    
    print("coordinates and Threshold")
    matplotlib.pyplot.imshow(labels, cmap=new_cmap)
    matplotlib.pyplot.plot(xs, ys, 'r.')
    matplotlib.pyplot.axis('off')
    matplotlib.pyplot.show()
    
    # Create a list to hold contours for each valid label
  
    # Optionally, show the contours on the image
    if showing:
        imc = imgray3.copy()
        for contours in contour_list:
            for contour in contours:
                imc = cv2.drawContours(imc, [contour], -1, (1, 1, 1), 1)
        print("contours")
        matplotlib.pyplot.imshow(imc)
        matplotlib.pyplot.axis('off')
        matplotlib.pyplot.show()
        
    return(labels,IDs,Areas,coordinates,contour_list,new_cmap,xs,ys,Centers,Ferets,angles)

def TextOutput(fn,contour_list,IDs,Areas,coordinates,xs,ys,Centers,Ferets,angles,imlistCon,impath,csvSaveFolder):
    #una vez tiene los contornos hay que hacer un ajuste con una elipse par encontrar el feret majory menor.
    #para una elipse se ncesitan al menos 6 puntos.
    DF=pandas.DataFrame({"ID":IDs,
                         "Xcoord":xs,
                         "Ycoord":ys,
                         "Center":Centers,
                         "Area[px]":Areas, 
                         "Feret [px]": Ferets,
                         "Angles":angles})
    os.makedirs(imlistCon[fn].replace(impath,csvSaveFolder).replace("\\","/").replace((imlistCon[fn].split("\\"))[-1],""), exist_ok=True)
    DF.to_csv((imlistCon[fn].replace(impath,csvSaveFolder).replace("\\","/"))+".csv")
    
    return(DF)

def ThreshoBlob(imgray,imlistCon,fn,thr_value2,max_area,edge_margin,showing,ImageSaveFolder):
    if showing==True:
        print("thresholded image for blob detection")
    blur = cv2.GaussianBlur(imgray,(5,5),0)
    ret3,thresh = cv2.threshold(blur,thr_value2,255,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)#
    kernel = numpy.ones((5, 5), numpy.uint8)  # Kernel for dilation/erosion
    #thresh = cv2.erode(thresh, kernel, iterations=1)  # Erosion to reduce noise
    #thresh = cv2.dilate(thresh, kernel, iterations=1)  # Dilation to expand blobs
    Showim(thresh,"/thresholdBlobs-"+imlistCon[fn],showing,ImageSaveFolder)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_image = numpy.zeros_like(imgray)
    for contour in contours:
        area = cv2.contourArea(contour)  # Calculate the area of the contour
        if area > max_area:
            cv2.drawContours(filtered_image, [contour], -1, (255), thickness=cv2.FILLED)
    if showing==True:
        print("Bigblobs to filter out")
    filtered_image = cv2.dilate(filtered_image, kernel, iterations=1)
    filtered_image = cv2.bitwise_not(filtered_image)
    thresh_with_border = numpy.zeros_like(thresh)
    thresh_with_border[edge_margin:-edge_margin, edge_margin:-edge_margin] = filtered_image[edge_margin:-edge_margin, edge_margin:-edge_margin]
    filtered_image=thresh_with_border
    Showim(filtered_image, "/filteredBlobs-" + imlistCon[fn], showing, ImageSaveFolder)
    
    return(thresh, filtered_image)

def AOIdetector(filtered_image,im2,imn2,edge_margin,Grain=False,showing=True,ImageSaveFolder=""):
    #Process image 2
    im2gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY) #threshold
    if showing == True:
        print("Gray")
        Showim(im2gray,"/Bright"+imn2,showing,ImageSaveFolder)
        print("thresholded image for AOI detection")
    blur = cv2.GaussianBlur(im2gray,(5,5),0)
    ret3,thresh2 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)#
    kernel = numpy.ones((5, 5), numpy.uint8)  # Kernel for dilation/erosion
    if Grain==True:
        thresh2 = cv2.bitwise_not(thresh2)
    thresh_with_border = numpy.zeros_like(thresh2)
    thresh_with_border[edge_margin:-edge_margin, edge_margin:-edge_margin] = thresh2[edge_margin:-edge_margin, edge_margin:-edge_margin]
    thresh2=thresh_with_border
    Showim(thresh2,"/thresholdBright-"+imn2,showing,ImageSaveFolder)
    contours, _ = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_image2 = numpy.zeros_like(im2gray)
    for contour in contours:
        area = cv2.contourArea(contour)  # Calculate the area of the contour
        if area > max_area:
            cv2.drawContours(filtered_image2, [contour], -1, (255), thickness=cv2.FILLED)
    if showing==True:
        print("previous filtered image")
    Showim(filtered_image, "", showing, ImageSaveFolder)
    if showing==True:
        print("AOI")
    #filtered_image = cv2.dilate(filtered_image, kernel, iterations=1)

    filtered_image2 = cv2.bitwise_and(filtered_image2,filtered_image)
    Showim(filtered_image2, "/AOI-" + imn2, showing, ImageSaveFolder)
    
    return(thresh2, filtered_image2)

def check_image_quality(image, blur_threshold=100, contrast_threshold=50):

    # 1. Check blurriness using the Laplacian variance
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    variance = laplacian.var()  # Calculate variance of the Laplacian
    
    # 2. Check contrast using the standard deviation of pixel intensities
    contrast = image.std()  # Standard deviation is a measure of contrast

    # Print the metrics (optional)
    print(f"Laplacian variance (blurriness): {variance}")
    print(f"Image contrast (std deviation): {contrast}")
    
    # Check if the image is blurry and low contrast
    if variance < blur_threshold:
        print("The image is too blurry.")
        return False
    elif contrast < contrast_threshold:
        print("The image has low contrast.")
        return False
    else:
        print("The image has good sharpness and contrast.")
        return True


def has_been_analyzed(impath, csvSaveFolder, fn, imlistCon):
    csv_filename = imlistCon[fn].replace(impath,csvSaveFolder).replace("\\","/")+".csv"
    return(os.path.exists(csv_filename))

def voronoi_diagram(centroids, save_path=None, showing=False):
    """
    Create and visualize a Voronoi diagram based on a list of centroids.

    Parameters:
        centroids (list of tuples): List of (x, y) coordinates for the centroids.
        save_path (str, optional): Path to save the Voronoi diagram image. Defaults to None.
        show (bool, optional): If True, displays the plot. Defaults to False.
    
    Returns:
        None
    """
    # Convert centroids to numpy array for Voronoi
    points = numpy.array(centroids)
    try:
        # Create the Voronoi diagram
        vor = Voronoi(points)

        # Plot the Voronoi diagram
        fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange', line_width=2, line_alpha=0.6)

        # Optionally save the plot
        if save_path:
            matplotlib.pyplot.savefig(save_path)

        # Show the plot
        if showing==True:
            matplotlib.pyplot.show()
        else:
            matplotlib.pyplot.close()
    except ValueError:
        print("ValueError in Voronoi generation")
        print(centroids)


def ripleys_k_function(centroids, r_max=600, num_bins=100, show=False, save_path=None):
    """
    Calculate and plot Ripley's K-function for a given set of points.

    Parameters:
        centroids (list of tuples): List of (x, y) coordinates for the centroids.
        r_max (int, optional): Maximum distance for the K-function analysis. Defaults to 600.
        num_bins (int, optional): Number of distance bins for the K-function calculation. Defaults to 100.
        show (bool, optional): If True, displays the plot. Defaults to False.
        save_path (str, optional): Path to save the Ripley's K-function plot. Defaults to None.
    
    Returns:
        None
    """
    # Convert centroids to numpy array
    points = numpy.array(centroids)

    # Calculate pairwise distances between points
    distances = cdist(points, points)

    # Define r-values for the K-function calculation
    r_values = numpy.linspace(0, r_max, num=num_bins)

    # Initialize the K-values array
    k_values = numpy.zeros(len(r_values))

    # Loop through r-values to calculate K(r)
    n = len(points)
    for i, r in enumerate(r_values):
        # Count the number of pairs of points that are within distance r (excluding self-pairs)
        k_values[i] = numpy.sum(distances < r) / n

    # Plot Ripley's K-function
    matplotlib.pyplot.plot(r_values, k_values, marker='o', label="K(r)")
    matplotlib.pyplot.axline((0, 0), slope=1, color='red', linestyle='--', label="$K(r) = r$")  # Reference line for random distribution
    matplotlib.pyplot.xlabel('Distance')
    matplotlib.pyplot.ylabel('Ripley\'s K-function')
    matplotlib.pyplot.legend()

    # Optionally save the plot
    if save_path:
        matplotlib.pyplot.savefig(save_path)

    # Show the plot
    if show:
        matplotlib.pyplot.show()
    else:
        matplotlib.pyplot.close()



def morans_i(centroids, values):
    """
    Calculate Moran's I for spatial autocorrelation.

    Parameters:
        centroids (list of tuples): List of (x, y) coordinates for the centroids.
        values (list): List of values associated with each centroid (e.g., colony sizes).

    Returns:
        morans_i_value (float): The Moran's I value.
        interpretation (str): Interpretation of Moran's I (Clustered, Dispersed, or Random).
    """
    # Convert centroids to GeoDataFrame
    geometry = geopandas.GeoSeries([Point(x, y) for x, y in centroids])  # Use shapely.geometry.Point here
    gdf = geopandas.GeoDataFrame({'value': values}, geometry=geometry)
    #gdf.set_geometry('geometry', inplace=True)
    # Create a spatial weights matrix using Queen contiguity
    try:
        w = Queen.from_dataframe(gdf)
            # Calculate Moran's I
        moran = Moran(gdf['value'], w)
        morans_i_value = moran.I

        # Interpret Moran's I
        if morans_i_value > 0:
            interpretation = "Spatial Clustering"
        elif morans_i_value < 0:
            interpretation = "Spatial Dispersion"
        else:
            interpretation = "Random Spatial Pattern"
    except KeyError:
        print(gdf)
        morans_i_value = None
        interpretation = "Error in Moran's calculation"
    return morans_i_value, interpretation

def calculate_geary_c_from_centroids_and_ferets(centroids, ferets):
    """
    Function to calculate Geary's C for spatial autocorrelation based on centroids and their minor Feret diameters.

    Parameters:
        centroids (list of tuples): List of (x, y) coordinates of the centroids.
        ferets (list of tuples): List of (major, minor) Feret diameters for each centroid.

    Returns:
        Geary's C value and interpretation.
    """
    try:
        # Ensure the centroids and ferets lists are the same length
        if len(centroids) != len(ferets):
            raise ValueError("The number of centroids must match the number of Feret values.")

        # Create a GeoDataFrame from the centroids
        geometry = geopandas.GeoSeries([Point(x, y) for x, y in centroids])
        
        # Extract the minor Feret values (second element of the tuple) as the spatial attribute
        minor_ferets = [minor for _, minor in ferets]

        # Create a GeoDataFrame with minor Feret values as the 'radius' attribute
        gdf = geopandas.GeoDataFrame({'minor_feret': minor_ferets}, geometry=geometry)
        gdf.set_geometry('geometry', inplace=True)
        # Create spatial weights using Queen contiguity
        #w = Queen.from_dataframe(gdf)
        w = Queen(gdf)

        # Calculate Geary's C for the minor Feret diameters
        geary = Geary(gdf['minor_feret'], w)

        # Interpret the Geary's C result
        if geary.C < 0.5:
            geary_interpretation = "Spatial Clustering"
        elif geary.C > 0.5:
            geary_interpretation = "Spatial Dispersion"
        else:
            geary_interpretation = "Random Spatial Pattern"

        # Return the result and interpretation
        return geary.C, geary_interpretation

    except Exception as e:
        print("Error in Geary's C calculation:", e)
        return None, None


def euclidean_distance(point1, point2):
    return numpy.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Nearest Neighbor Search using KDTree
def nearest_neighbor_kd_tree(centroids):
    # Build a KD-tree for fast nearest neighbor search
    tree = cKDTree(centroids)

    nearest_neighbors = []
    for i, centroid in enumerate(centroids):
        # Query the nearest neighbor (k=2 because the closest point is itself, so we ask for 2 neighbors)
        dist, index = tree.query(centroid, k=2)  # k=2 to find the closest neighbor excluding itself
        nearest_neighbors.append((i, index[1], dist[1]))  # Store the nearest neighbor's index and distance

    return(nearest_neighbors)


import numpy as np

def estadisticas(datos):
    """
    Calcula el promedio, desviación estándar, número de elementos (N), 
    error estándar de la media (SEM) y número de posibles outliers en una lista de valores.

    Parámetros:
    datos (list): Lista de valores numéricos para calcular las estadísticas.

    Retorna:
    promedio (float): El promedio de los datos.
    desviacion_estandar (float): La desviación estándar de los datos.
    N (int): Número de elementos en la lista.
    sem (float): Error estándar de la media.
    num_outliers (int): Número de outliers en los datos.
    outliers (list): Lista de los valores considerados outliers.
    """
    
    # Verificar que los datos no estén vacíos
    if len(datos) == 0:
        return "La lista de datos está vacía"

    # Convertir la lista de datos en un arreglo de NumPy para facilitar los cálculos
    datos_array = numpy.array(datos)

    # Cálculos básicos
    promedio = numpy.mean(datos_array)
    desviacion_estandar = numpy.std(datos_array)
    N = len(datos_array)
    sem = desviacion_estandar / numpy.sqrt(N)

    # Cálculo de posibles outliers usando el criterio del rango intercuartil (IQR)
    Q1 = numpy.percentile(datos_array, 25)
    Q3 = numpy.percentile(datos_array, 75)
    IQR = Q3 - Q1

    # Definición de los límites para los outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Contar el número de outliers
    outliers = [x for x in datos_array if x < limite_inferior or x > limite_superior]
    num_outliers = len(outliers)

    # Devolver los resultados como variables separadas
    return promedio, desviacion_estandar, N, sem, num_outliers, outliers


def analyze_area_coverage(filtered_image, Areas):
    """
    Calculate the percentage of the analyzed area covered by colonies and the total area in pixels.
    
    Parameters:
    filtered_image (ndarray): Binary (thresholded) image where the analyzed area is marked.
    Areas (list): List of areas of individual colonies.
    
    Returns:
    float: Percentage of analyzed area covered by colonies.
    int: Total analyzed area in pixels.
    """
    # Calculate the total area of colonies from the list Areas
    total_colony_area = sum(Areas)
    
    # Calculate the total analyzed area in pixels (number of non-zero pixels in the filtered image)
    analyzed_area_pixels = numpy.sum(filtered_image > 0)
    
    # Calculate the percentage of colonies covering the analyzed area
    if analyzed_area_pixels > 0:
        coverage_percentage = (total_colony_area / analyzed_area_pixels) * 100
    else:
        coverage_percentage = 0
    
    return coverage_percentage, analyzed_area_pixels

In [ ]:
def runSingle(
    impath,
    labelsD,
    labelR,
    imlabel,
    BadLabel,
    ImageSaveFolder,
    csvSaveFolder,
    Grain,
    blur_threshold=100, 
    contrast_threshold=50,
    fn=0,
    label="DAPI",
    Cond="",
    edge_margin=10,
    rad=50,
    thr_value2=100,
    max_area=50*50*numpy.pi,
    size_value=2,
    min_distance_value=2,
    threshold_abs_value=0.1,
    Blocksize=15,
    Constant=-1,
    min_thresh_counter=4,
    showing=True,
    QualityCheck=False,
    Spatial=False):
    Conds=[]
    for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
        Conds += [Folder]
    imlist=[]
    imlist2=[]
    for (root,dirs,files) in os.walk(impath, topdown=True):
        for file in files:
            if not file.endswith(imlabel):
                continue
            if label in file:
                imlist += [os.path.join(root,file)]
            if labelR in file:
                imlist2 += [os.path.join(root,file)]
    imlistCon=[f for f in imlist if Cond in f]
    imlist2Con=[f for f in imlist2 if Cond in f]
    imn=imlistCon[fn]
    im=matplotlib.pyplot.imread(imn)
    Showim(im,imlistCon[fn],showing,ImageSaveFolder)
    #Process image 1
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) #threshold
    if showing == True:
        print("Gray")
        Showim(imgray,"/Dye"+imlistCon[fn],showing,ImageSaveFolder)
    imgray2=imgray
    if QualityCheck==True:
        quality=check_image_quality(imgray2, blur_threshold, contrast_threshold)
    else:
        quality=True
    if quality==True:
        imgray,imgray2,imgray3=backgroundrolling(im,rad,imlistCon,fn,showing,ImageSaveFolder="")
        thresh,filtered_image=ThreshoBlob(imgray3,imlistCon,fn,thr_value2,max_area,edge_margin,showing,ImageSaveFolder)
        if labelR != "" and len(imlistCon)==len(imlist2Con):
            imn2=imlist2Con[fn]
            im2=matplotlib.pyplot.imread(imn2)
            Showim(im2,imlist2Con[fn],showing,ImageSaveFolder)
            thresh2,filtered_image=AOIdetector(filtered_image,im2,imn2,edge_margin,Grain,showing,ImageSaveFolder)
        image_max,coordinates = FindLocalMax(imgray2,size_value,min_distance_value,threshold_abs_value,edge_margin,showing)
        thresh=threshold(imgray2,imlistCon,fn,thr_value2,Blocksize,Constant,edge_margin,filtered_image,showing,ImageSaveFolder)
        labels,IDs,Areas,coordinates,contour_list,new_cmap,xs,ys,Centers,Ferets,angles=coords(imlistCon,fn,thresh,coordinates,imgray3,min_thresh_counter,showing,ImageSaveFolder)
        nearest_neighbors=nearest_neighbor_kd_tree(Centers)
        if Spatial==True:
            geary, geary_interpretation=calculate_geary_c_from_centroids_and_ferets(Centers,Ferets)
            voronoi_diagram(Centers, ImageSaveFolder, showing)
            morans_i_value, Moran_interpretation= morans_i(Centers, Areas)
            ripleys_k_function(Centers, 600, 100, showing, ImageSaveFolder)
        else:
            geary, geary_interpretation,morans_i_value, Moran_interpretation = None, "None", None, "None"
        AreaperFieldporcent, AnalyzedArea=analyze_area_coverage(filtered_image, Areas)
        avAreas,STDAreas,NAreas,_,_,_=estadisticas(Areas)
        avMferet,STDMferet,NMferet,_,_,_=estadisticas([feret[0] for feret in Ferets])
        avmferet,STDmferet,Nmferet,_,_,_=estadisticas([feret[1] for feret in Ferets])
        avAng,STDAng,NAng,_,_,_=estadisticas(angles)
        avNN,STDNN,NNN,_,_,_=estadisticas([nn[2] for nn in nearest_neighbors])
        DF=TextOutput(fn,contour_list,IDs,Areas,coordinates,xs,ys,Centers,Ferets,angles,imlistCon,impath,csvSaveFolder)
        DF2=pandas.DataFrame({"ID":[imn.split("/")[-1]],
                            "Area per colony Average [px]": [avAreas],
                            "Area per colony Standard deviation [px]": [STDAreas],
                            "Number of colonies": [NAreas],
                            "Major Feret Average [px]": [avMferet],
                            "Major Feret Standard deviation [px]": [STDMferet],
                            "Minor Feret Average [px]": [avmferet],
                            "Minor Feret Standard deviation": [STDmferet],
                            "Angles Average": [avAng],
                            "Angles Standard deviation": [STDAng],
                            "Nearest neighbor Average [px]": [avNN],
                            "Nearest neighbor Standard deviation [px]": [STDNN],
                            "Area Colonized [%]": [AreaperFieldporcent],
                            "Area Analyzed [px]": [AnalyzedArea],
                            "Geary's C": [geary],
                            "Geary's C interpetration": [geary_interpretation],
                            "Moran's I": [morans_i_value],
                            "Moran's I interpetration": [Moran_interpretation],
                            "Ripley's K": [ripleys_k_function]})
        #os.makedirs(imlistCon[fn].replace(impath,csvSaveFolder).replace("\\","/").replace((imlistCon[fn].split("\\"))[-1],""), exist_ok=True)
        file_path = os.path.join(csvSaveFolder, Cond)+".csv"
        if os.path.exists(file_path):
            existing_df = pandas.read_csv(file_path)
            combined_df = pandas.concat([existing_df, DF2], ignore_index=True)
            combined_df.to_csv(file_path, index=False)
        else:
            DF2.to_csv(file_path, index=False)
        print(DF2)
        return(DF2)

In [ ]:
def runMany(
    impath,
    labelsD,
    labelR,
    imlabel,
    BadLabel,
    ImageSaveFolder,
    csvSaveFolder,
    Cond="",
    Grain=False,
    blur_threshold=100, 
    max_area=50*50*numpy.pi,
    contrast_threshold=50,
    edge_margin=10,
    rad=50,
    thr_value2=100,
    size_value=2,
    min_distance_value=2,
    min_thresh_counter=4,
    threshold_abs_value=0.1,
    Blocksize=15,
    Constant=-1,
    showing=True,
    QualityCheck=False,
    Reverse=True,
    Spatial=True
):
    if Cond=="":
        Conds=[]
        for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
            Conds += [Folder]
    else:
        Conds=[Cond]
    imlist=[]
    imlist2=[]
    for label in labelsD:
        if Reverse==True:
            Conds.reverse
        for Cond in Conds:
            for (root,dirs,files) in os.walk(impath, topdown=True):
                for file in files:
                    if not file.endswith(imlabel):
                        continue
                    if label in file:
                        imlist += [os.path.join(root,file)]
                    if labelR in file:
                        imlist2 += [os.path.join(root,file)]
            imlistCon=[f for f in imlist if Cond in f]
            imlist2Con=[f for f in imlist2 if Cond in f]
            if Reverse==True:
                imlistCon.reverse
                imlist2Con.reverse
            for fn in range(len(imlistCon)):
                print(str(fn)+"/"+str(len(imlistCon)))
                # Verificar si la imagen ya ha sido analizada antes de comenzar
                if has_been_analyzed(impath, csvSaveFolder, fn, imlistCon) == True:
                    print(f"El archivo {imlistCon[fn]} ya ha sido analizado. Omite el análisis.")
                    continue  # Omite el procesamiento de esta imagen
                imn=imlistCon[fn]
                im=matplotlib.pyplot.imread(imn)
                if showing==True:
                    print("Starting analysis of image: "+imn)
                    Showim(im,imlistCon[fn],showing,ImageSaveFolder)
                #Process image 1
                imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) #threshold
                if showing == True:
                    print("Gray")
                    Showim(imgray,"/Dye"+imlistCon[fn],showing,ImageSaveFolder)
                imgray2=imgray
                if QualityCheck==True:
                    quality=check_image_quality(imgray2, blur_threshold, contrast_threshold)
                else:
                    quality=True
                if quality==True:
                    imgray,imgray2,imgray3=backgroundrolling(im,rad,imlistCon,fn,showing,ImageSaveFolder="")
                    thresh,filtered_image=ThreshoBlob(imgray3,imlistCon,fn,thr_value2,max_area,edge_margin,showing,ImageSaveFolder)
                    if labelR != "" and len(imlistCon)==len(imlist2Con):
                        imn2=imlist2Con[fn]
                        im2=matplotlib.pyplot.imread(imn2)
                        Showim(im2,imlist2Con[fn],showing,ImageSaveFolder)
                        thresh2,filtered_image=AOIdetector(filtered_image,im2,imn2,edge_margin,Grain,showing,ImageSaveFolder)
                    image_max,coordinates = FindLocalMax(imgray2,size_value,min_distance_value,threshold_abs_value,edge_margin,showing)
                    thresh=threshold(imgray2,imlistCon,fn,thr_value2,Blocksize,Constant,edge_margin,filtered_image,showing,ImageSaveFolder)
                    labels,IDs,Areas,coordinates,contour_list,new_cmap,xs,ys,Centers,Ferets,angles=coords(imlistCon,fn,thresh,coordinates,imgray3,min_thresh_counter,showing,ImageSaveFolder)
                    nearest_neighbors=nearest_neighbor_kd_tree(Centers)
                    if spatial==True:
                        geary, geary_interpretation=calculate_geary_c_from_centroids_and_ferets(Centers,Ferets)
                        voronoi_diagram(Centers, ImageSaveFolder, showing)
                        morans_i_value, Moran_interpretation= morans_i(Centers, Areas)
                        ripleys_k_function(Centers, 600, 100, showing, ImageSaveFolder)
                    else:
                        geary, geary_interpretation,morans_i_value, Moran_interpretation = None, "None", None, "None"
                    AreaperFieldporcent, AnalyzedArea=analyze_area_coverage(filtered_image, Areas)
                    avAreas,STDAreas,NAreas,_,_,_=estadisticas(Areas)
                    avMferet,STDMferet,NMferet,_,_,_=estadisticas([feret[0] for feret in Ferets])
                    avmferet,STDmferet,Nmferet,_,_,_=estadisticas([feret[1] for feret in Ferets])
                    avAng,STDAng,NAng,_,_,_=estadisticas(angles)
                    avNN,STDNN,NNN,_,_,_=estadisticas([nn[2] for nn in nearest_neighbors])
                    DF=TextOutput(fn,contour_list,IDs,Areas,coordinates,xs,ys,Centers,Ferets,angles,imlistCon,impath,csvSaveFolder)
                    DF2=pandas.DataFrame({"ID":[imn.split("/")[-1]],
                                        "Area per colony Average [px]": [avAreas],
                                        "Area per colony Standard deviation [px]": [STDAreas],
                                        "Number of colonies": [NAreas],
                                        "Major Feret Average [px]": [avMferet],
                                        "Major Feret Standard deviation [px]": [STDMferet],
                                        "Minor Feret Average [px]": [avmferet],
                                        "Minor Feret Standard deviation": [STDmferet],
                                        "Angles Average": [avAng],
                                        "Angles Standard deviation": [STDAng],
                                        "Nearest neighbor Average [px]": [avNN],
                                        "Nearest neighbor Standard deviation [px]": [STDNN],
                                        "Area Colonized [%]": [AreaperFieldporcent],
                                        "Area Analyzed [px]": [AnalyzedArea],
                                        "Geary's C": [geary],
                                        "Geary's C interpetration": [geary_interpretation],
                                        "Moran's I": [morans_i_value],
                                        "Moran's I interpetration": [Moran_interpretation],
                                        "Ripley's K": [ripleys_k_function]})
                    #os.makedirs(imlistCon[fn].replace(impath,csvSaveFolder).replace("\\","/").replace((imlistCon[fn].split("\\"))[-1],""), exist_ok=True)
                    file_path = os.path.join(csvSaveFolder, Cond)+".csv"
                    if os.path.exists(file_path):
                        existing_df = pandas.read_csv(file_path)
                        combined_df = pandas.concat([existing_df, DF2], ignore_index=True)
                        combined_df.to_csv(file_path, index=False)
                        DF2=combined_df
                    else:
                        DF2.to_csv(file_path, index=False)
            
            file_path = os.path.join(csvSaveFolder, Cond)+".csv"
            if os.path.exists(file_path):
                existing_df = pandas.read_csv(file_path)
                DF2=existing_df
    return(DF2)

In [ ]:
# Función para calcular el coeficiente de variación
def coeficiente_de_variacion(datos):
    """
    Calcula el Coeficiente de Variación (CV) de una lista de datos.
    El Coeficiente de Variación se calcula como (desviación estándar / media) * 100.
    
    :param datos: Lista de datos numéricos
    :return: Coeficiente de variación
    """
    media = numpy.mean(datos)
    desviacion_estandar = numpy.std(datos)
    
    if media != 0:
        cv = (desviacion_estandar / media) * 100
    else:
        cv = 0  # Si la media es 0, el CV es 0 (caso límite)
    
    return cv


# Asegurarse de que la columna 'Colony Density [mm^-2]' exista en el CSV
def List_CV(csv_file,namecol):
    csv = pandas.read_csv(csv_file)
    try:
        if namecol in csv.columns:
            #print(csv)
            return csv[namecol].dropna().tolist()
        else:
            return []
    except pd.errors.EmptyDataError:
        print(f"Error: El archivo {csv_file} está vacío.")
        return []
    except Exception as e:
        print(f"Error al leer el archivo {csv_file}: {e}")
        return []



def CV_rep(datos, csv_file, size=2, groups=3, reps=25):
    datos=List_CV(csv_file,"Number of colonies")
    if not len(datos) >= size*groups:
        print("not enough data")
        return[]
    else:
        CVs=[]
        for x in range(reps):
            #make groups of size to calculate CV.
            ranNums = random.sample(datos, size*groups)
            #print(ranNums)
            mean_group=[]
            for n in range(groups):
                group=ranNums[(n)*size:(n+1)*size]
                #print(ranNums[(n)*size:(n+1)*size])
                #Calcular promedio y std de cada grupo de datos
                mean_group.append(statistics.mean(group))
            #calcula CV
            mean_dat=statistics.mean(mean_group)
            std_dat=statistics.stdev(mean_group)
            CV_dat=(std_dat/mean_dat)*100
            CVs.append(CV_dat)
            #print(str(x+1)+": "+str(CV_dat))
        return(CVs)

def CV(Cond,csvSaveFolder,dana="Number of colonies",groups=3,reps=25):
    print(Cond)
    csv_file = os.path.join(csvSaveFolder, Cond)+".csv"
    sizes=[1,2,3,6,9,12,18,27,30,36,72,96,120]
    datos=List_CV(csv_file,dana)
    #print(len(datos))
    #print(datos)
    cv_dict = {}
    for size in sizes:
        print("size: "+str(size))
        CV_dat=CV_rep(datos,csv_file,size, groups, reps)
        if not CV_dat==[]:
            cv_dict[size] = CV_dat
    saveCV=os.path.join(csvSaveFolder,"CV-"+dana+"-"+Cond)
    pandas.DataFrame(cv_dict).to_csv(saveCV, index=False)
    print()

In [ ]:
def save_parameters_to_csv_dynamic(
    labelsD, labelR, imlabel, BadLabel,
    impath, ImageSaveFolder, csvSaveFolder,
    pixeltoum, AreaperField, squaredPixel,
    intersection_threshold, x, showing,
    minArea, maxArea, cheackarea, Grain,
    thr_value2, min_thresh_counter, max_area,
    edge_margin, rad, size_value, min_distance_value,
    threshold_abs_value, Blocksize, Constant,
    QualityCheck, blur_threshold, contrast_threshold
):
    # Create directories if they don't exist
    if not os.path.exists(ImageSaveFolder):
        os.makedirs(ImageSaveFolder)
    if not os.path.exists(csvSaveFolder):
        os.makedirs(csvSaveFolder)

    # Create the parameters dictionary
    params = {
        "labelsD": labelsD,
        "labelR": labelR,
        "imlabel": imlabel,
        "BadLabel": BadLabel,
        "impath": impath,
        "ImageSaveFolder": ImageSaveFolder,
        "csvSaveFolder": csvSaveFolder,
        "pixeltoum": pixeltoum,
        "AreaperField": AreaperField,
        "squaredPixel": squaredPixel,
        "intersection_threshold": intersection_threshold,
        "x": x,
        "showing": showing,
        "minArea": minArea,
        "maxArea": maxArea,
        "cheackarea": cheackarea,
        "Grain": Grain,
        "thr_value2": thr_value2,
        "min_thresh_counter": min_thresh_counter,
        "max_area": max_area,
        "edge_margin": edge_margin,
        "rad": rad,
        "size_value": size_value,
        "min_distance_value": min_distance_value,
        "threshold_abs_value": threshold_abs_value,
        "Blocksize": Blocksize,
        "Constant": Constant,
        "QualityCheck": QualityCheck,
        "blur_threshold": blur_threshold,
        "contrast_threshold": contrast_threshold
    }

    # Set CSV file path
    csv_file = os.path.join(csvSaveFolder, "parameters.csv")

    # Save parameters to CSV
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(params.keys())    # Header
        writer.writerow(params.values())  # Data

    print(f"Parameters saved to {csv_file}")

# Parameters

In [ ]:
labelsD=["DAPI"]
labelR=""
imlabel=".tif"
BadLabel="ORG"
impath=""
ImageSaveFolder=""
csvSaveFolder=""

if os.path.exists(ImageSaveFolder)==False:
    os.makedirs(ImageSaveFolder)
if os.path.exists(csvSaveFolder)==False:
    os.makedirs(csvSaveFolder)
x=0
ntersection_threshold = 0.5
showing=True
minArea=(2**2)*math.pi
maxArea=math.pi*10**2
cheackarea=True
Grain=False
thr_value2 = 100
min_thresh_counter=4
max_area=maxArea
edge_margin=15
rad=50
size_value=2
min_distance_value=2 
threshold_abs_value=0.02
Blocksize=15
Constant=-1
intersection_threshold = 0.5  # Ajusta este valor según tu necesidad
QualityCheck=False
blur_threshold=100
contrast_threshold=5
showing=True
Spatial=True

Conds=[]
for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
    Conds += [Folder]

label=labelsD[0]
Cond=Conds[0]
fn=0

# Interact for parameters

In [ ]:
matplotlib.pyplot.rcParams['figure.figsize'] = [50, 25]

In [ ]:
interact_manual(runSingle,
        impath=fixed(impath),
        labelsD=fixed(labelsD),
        labelR=fixed(labelR),
        imlabel=fixed(imlabel),
        BadLabel=fixed(BadLabel),
        ImageSaveFolder=fixed(ImageSaveFolder),
        csvSaveFolder=fixed(csvSaveFolder),
        Grain=Grain,
        fn=fixed(fn),
        label=fixed(label),
        Cond=fixed(Cond),
        blur_threshold=blur_threshold,
        contrast_threshold=contrast_threshold,
        edge_margin=edge_margin,
        rad=rad,
        thr_value2=thr_value2,
        max_area=max_area,
        size_value=size_value,
        min_distance_value=min_distance_value,
        threshold_abs_value=threshold_abs_value,
        Blocksize=Blocksize,
        Constant=Constant,
        min_thresh_counter=min_thresh_counter,
        QualityCheck=QualityCheck,
        showing=fixed(True)
        )

In [ ]:
save_parameters_to_csv_dynamic(
    labelsD, labelR, imlabel, BadLabel,
    impath, ImageSaveFolder, csvSaveFolder,
    pixeltoum, AreaperField, squaredPixel,
    intersection_threshold, x, showing,
    minArea, maxArea, cheackarea, Grain,
    thr_value2, min_thresh_counter, max_area,
    edge_margin, rad, size_value, min_distance_value,
    threshold_abs_value, Blocksize, Constant,
    QualityCheck, blur_threshold, contrast_threshold)

In [ ]:
runSingle(impath=impath,
        labelsD=labelsD,
        labelR=labelR,
        imlabel=imlabel,
        BadLabel=BadLabel,
        ImageSaveFolder=ImageSaveFolder,
        csvSaveFolder=csvSaveFolder,
        Grain=Grain,
        fn=fn,
        label=label,
        Cond=Cond,
        blur_threshold=blur_threshold,
        contrast_threshold=contrast_threshold,
        edge_margin=edge_margin,
        rad=rad,
        thr_value2=thr_value2,
        max_area=max_area,
        size_value=size_value,
        min_distance_value=min_distance_value,
        threshold_abs_value=threshold_abs_value,
        Blocksize=Blocksize,
        Constant=Constant,
        min_thresh_counter=min_thresh_counter,
        QualityCheck=QualityCheck,
        showing=True,
        Spatial=True)

# run

In [ ]:
runMany(
    impath,
    labelsD,
    labelR,
    imlabel,
    BadLabel,
    ImageSaveFolder,
    csvSaveFolder,
    
    Cond=Conds[3],
    Grain=Grain,
    blur_threshold=blur_threshold, 
    max_area=max_area,
    contrast_threshold=contrast_threshold,
    edge_margin=edge_margin,
    rad=rad,
    thr_value2=thr_value2,
    size_value=size_value,
    min_distance_value=min_distance_value,
    min_thresh_counter=min_thresh_counter,
    threshold_abs_value=threshold_abs_value,
    Blocksize=Blocksize,
    Constant=Constant,
    showing=False,
    QualityCheck=QualityCheck,
    Reverse=True,
    Spatial=Spatial
)

In [ ]:
for Cond in Conds:
    print(Cond)
    CV(Cond,csvSaveFolder,"Number of colonies",3,25)
    CV(Cond,csvSaveFolder,"Area Colonized [%]",3,25)

In [ ]:
# Directory containing the CSV files
input_directory = csvSaveFolder  # Replace with your directory path

filename=[filename for filename in os.listdir(csvSaveFolder) if filename.endswith(".csv")][0]
cols=pandas.read_csv(os.path.join(csvSaveFolder,filename)).columns
for col in cols:
    condition_dfs=[]
    for Con in Conds:
        for filename in os.listdir(input_directory):
            if filename.endswith(".csv") and Cond in filename:
                # Read the CSV file
                file_path = os.path.join(csvSaveFolder, filename)
                df = pandas.read_csv(file_path)
                # Extract the condition name from the filename (e.g., 'condition1.csv' -> 'condition1')
                condition_name = filename.replace("csvSaveFolder","").replace(".csv","")
                #print(condition_name)
                condition_dfs.append(df[[col]].rename(columns={col: condition_name}))
    # Merge all DataFrames on the index (assuming all CSVs have the same index structure)
    merged_df = pandas.concat(condition_dfs, axis=1)

    # Save the merged DataFrame to a new CSV
    output_file = os.path.join(csvSaveFolder,col+".csv")  # Desired output file name
    merged_df.to_csv(output_file, index=False)
    print(merged_df)
    print(f"Merged CSV saved as {output_file}")

# Parallel run

In [ ]:
#Ejecuta una sola vez
i = 0
totC = 10
i*totC+totC

In [ ]:
torun=range(len(Conds))

In [ ]:
#FoldersC son las carpetas a ejecutar en cluster paralelos.
running = torun[i*totC:i*totC+totC]
totC = len(running)
i = i+1
running

In [ ]:
# Abre Clusters paralelos
import ipyparallel as ipp
cluster = ipp.Cluster(n=totC)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=totC)
print(rc.ids)

In [ ]:
with rc[:].sync_imports():
    import csv
    import cv2
    import geopandas
    import math
    import matplotlib
    import matplotlib.pyplot
    import numpy
    import os
    import pandas

    from esda import Moran, Geary
    from ipywidgets import interact, interactive, fixed, interact_manual
    from libpysal.weights import Queen
    from scipy import ndimage
    from scipy.spatial import cKDTree, Voronoi, voronoi_plot_2d,distance
    from scipy.spatial.distance import cdist
    from shapely.geometry import Point
    from skimage import restoration, data, img_as_float
    from skimage.feature import peak_local_max
    from skimage.segmentation import watershed
    %matplotlib inline

In [ ]:
# Define las funciones enc ada cluster
rc[:].push(dict(
    Showim=Showim,
    countArray=countArray,
    backgroundrolling=backgroundrolling,
    FindLocalMax=FindLocalMax,
    threshold=threshold,
    coords=coords,
    TextOutput=TextOutput,
    ThreshoBlob=ThreshoBlob,
    AOIdetector=AOIdetector,
    check_image_quality=check_image_quality,
    has_been_analyzed=has_been_analyzed,
    voronoi_diagram=voronoi_diagram,
    ripleys_k_function=ripleys_k_function,
    morans_i=morans_i,
    calculate_geary_c_from_centroids_and_ferets=calculate_geary_c_from_centroids_and_ferets,
    euclidean_distance=euclidean_distance,
    nearest_neighbor_kd_tree=nearest_neighbor_kd_tree,
    estadisticas=estadisticas,
    analyze_area_coverage=analyze_area_coverage,
    runMany=runMany))

In [ ]:
results = {}
for numb in range(len(running)):
    results[numb] = rc[numb].apply_async(runMany,
                                         impath, 
                                         labelsD, 
                                         labelR,
                                         imlabel,
                                         BadLabel,
                                         ImageSaveFolder,
                                         csvSaveFolder,
                                         Conds[numb],
                                         Grain,
                                         blur_threshold,
                                         max_area,
                                         contrast_threshold,
                                         edge_margin,
                                         rad,
                                         thr_value2,
                                         size_value,
                                         min_distance_value,
                                         min_thresh_counter,
                                         threshold_abs_value,
                                         Blocksize, 
                                         Constant,
                                         showing,
                                         QualityCheck,
                                         Spatial)#, True)

In [ ]:
#Indica si los resultados estan listos. ejecuta cuantas veces sea necesario
for numb in range(totC):
    print('is engine '+str(numb)+' ready?:')
    print(results[numb].ready())
    print('')

In [ ]:
#Revisa si ocurrio algun error
for num in range(totC):
    print('Engine '+str(num)+' ')
    try:
        results[num].get(1)
        print('Results ready without error')
        print('')
    except Exception as error:
        print(error)
        print('')